In [2]:
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, UpSampling2D, Concatenate, MaxPooling2D)
from tensorflow.keras.models import Model

def unet():
    """ set up unet model """
    inputs = Input(shape=(572, 572, 1), name='input')
    conv1_1 = Conv2D(64, 3, activation='relu', name='conv1_1')(inputs)
    conv1_2 = Conv2D(64,3,activation='relu',name='conv1_2')(conv1_1)

    max_pool1 = MaxPooling2D(name='maxpool1')(conv1_2) # maxpooling2D默认stride=filer size
    conv2_1 = Conv2D(128,3,activation='relu',name='conv2_1')(max_pool1)
    conv2_2 = Conv2D(128,3,activation='relu',name='conv2_2')(conv2_1)


    
    max_pool2 = MaxPooling2D(name='maxpool2')(conv2_2)
    conv3_1 = Conv2D(256,3,activation='relu',name='conv3_1')(max_pool2)
    conv3_2 = Conv2D(256,3,activation='relu',name='conv3_2')(conv3_1)
 
    maxpool3 = MaxPooling2D(name='maxpool3')(conv3_2)
    conv4_1 = Conv2D(512,3,activation='relu',name='conv4_1')(maxpool3)
    conv4_2 = Conv2D(512,3,activation='relu',name='conv4_2')(conv4_1)
 
    maxpool4 = MaxPooling2D(name='maxpool4')(conv4_2)
    conv5_1 = Conv2D(1024,3,activation='relu',name='conv5_1')(maxpool4)
    conv5_2 = Conv2D(1024,3,activation='relu',name='conv5_2')(conv5_1)
 
    up5 = UpSampling2D(name='up5')(conv5_2)
    up5_conv = Conv2D(512,2,padding='same',name='up5_conv')(up5)
    conv4_feature = tf.image.resize(conv4_2,(up5_conv.shape[1],up5_conv.shape[2]),
                                    tf.image.ResizeMethod.NEAREST_NEIGHBOR,
                                    name='conv4_feature')

    concat1 = Concatenate(name='concat1')([up5_conv,conv4_feature])
    conv6_1 = Conv2D(512,3,activation='relu',name='conv6_1')(concat1)
    conv6_2 = Conv2D(512,3,activation='relu',name='conv6_2')(conv6_1)
 
    up6 = UpSampling2D(name='up6')(conv6_2)
    up6_conv = Conv2D(256,2,padding='same',name='up6_conv')(up6)
    conv3_feature = tf.image.resize(conv3_2,(up6_conv.shape[1],up6_conv.shape[2]),
                                    tf.image.ResizeMethod.NEAREST_NEIGHBOR,
                                    name='conv3_feature')
 
    concat2 = Concatenate(name='concat2')([up6_conv,conv3_feature])
    conv7_1 = Conv2D(256,3,activation='relu',name='conv7_1')(concat2)
    conv7_2 = Conv2D(256,3,activation='relu',name='conv7_2')(conv7_1)
 
    up7 = UpSampling2D(name='up7')(conv7_2)
    up7_conv = Conv2D(128,2,padding='same',name='up7_conv')(up7)
    conv2_feature = tf.image.resize(conv2_2,(up7_conv.shape[1],up7_conv.shape[2]),
                                    tf.image.ResizeMethod.NEAREST_NEIGHBOR,
                                    name='conv2_feature')
 
    concat3 = Concatenate(name='concat3')([up7_conv,conv2_feature])
    conv8_1 = Conv2D(128,3,activation='relu',name='conv8_1')(concat3)
    conv8_2 = Conv2D(128,3,activation='relu',name='conv8_2')(conv8_1)
 
    up8 = UpSampling2D(name='up8')(conv8_2)
    up8_conv = Conv2D(64,2,padding='same',name='up8_conv')(up8)
    conv1_feature = tf.image.resize(conv1_2,(up8_conv.shape[1],up8_conv.shape[2]),
                                    tf.image.ResizeMethod.NEAREST_NEIGHBOR,
                                    name='conv1_feature')
 
    concat4 = Concatenate(name='concat4')([up8_conv,conv1_feature])
    conv9_1 = Conv2D(64,3,activation='relu',name='conv9_1')(concat4)
    conv9_2 = Conv2D(64, 3, activation='relu', name='conv9_2')(conv9_1)
    out = Conv2D(2, 1, activation='sigmoid', name='out')(conv9_2)

    model = Model(inputs=inputs, outputs=out, name='mytf2_unet')

    return model

if __name__ == '__main__':
    model = unet()
    model.summary()

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class double_conv2d_bn(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=3,strides=1,padding=1):
        super(double_conv2d_bn,self).__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,
                               kernel_size=kernel_size,
                              stride = strides,
                              padding=padding,
                              bias=True)
        self.conv2 = nn.Conv2d(out_channels,out_channels,
                              kernel_size = kernel_size,
                              stride = strides,padding=padding,bias=True)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels) 
    
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out
    
class deconv2d_bn(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=2,strides=2):
        super(deconv2d_bn,self).__init__()
        self.conv1 = nn.ConvTranspose2d(in_channels,out_channels,
                                        kernel_size = kernel_size,
                                       stride = strides,bias=True)
        self.bn1 = nn.BatchNorm2d(out_channels)

    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        return out
    
class Unet(nn.Module):
    def __init__(self):
        super(Unet,self).__init__()
        self.layer1_conv = double_conv2d_bn(1,8)
        self.layer2_conv = double_conv2d_bn(8,16)
        self.layer3_conv = double_conv2d_bn(16,32)
        self.layer4_conv = double_conv2d_bn(32,64)
        self.layer5_conv = double_conv2d_bn(64,128)
        self.layer6_conv = double_conv2d_bn(128,64)
        self.layer7_conv = double_conv2d_bn(64,32)
        self.layer8_conv = double_conv2d_bn(32,16)
        self.layer9_conv = double_conv2d_bn(16,8)
        self.layer10_conv = nn.Conv2d(8,1,kernel_size=3,
                                     stride=1,padding=1,bias=True)
        
        self.deconv1 = deconv2d_bn(128,64)
        self.deconv2 = deconv2d_bn(64,32)
        self.deconv3 = deconv2d_bn(32,16)
        self.deconv4 = deconv2d_bn(16,8)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        conv1 = self.layer1_conv(x)
        pool1 = F.max_pool2d(conv1,2)
        
        conv2 = self.layer2_conv(pool1)
        pool2 = F.max_pool2d(conv2,2)
        
        conv3 = self.layer3_conv(pool2)
        pool3 = F.max_pool2d(conv3,2)
        
        conv4 = self.layer4_conv(pool3)
        pool4 = F.max_pool2d(conv4,2)
        
        conv5 = self.layer5_conv(pool4)
        
        convt1 = self.deconv1(conv5)
        concat1 = torch.cat([convt1,conv4],dim=1)
        conv6 = self.layer6_conv(concat1)
        
        convt2 = self.deconv2(conv6)
        concat2 = torch.cat([convt2,conv3],dim=1)
        conv7 = self.layer7_conv(concat2)
        
        convt3 = self.deconv3(conv7)
        concat3 = torch.cat([convt3,conv2],dim=1)
        conv8 = self.layer8_conv(concat3)
        
        convt4 = self.deconv4(conv8)
        concat4 = torch.cat([convt4,conv1],dim=1)
        conv9 = self.layer9_conv(concat4)
        outp = self.layer10_conv(conv9)
        outp = self.sigmoid(outp)
        return outp
    

model = Unet()
inp = torch.rand(10,1,224,224)
outp = model(inp)
print(outp.shape)